# 把結果轉成TXT檔, 以便更新資料庫

In [1]:
IP   = "10.120.26.4"
User = "iii"
PWd  = "iii"
DB   = "iii"
###　　　###　　　###　　　####
# [二] #
######　　　SQL命令　　　######
sql = "SELECT Attr_id,Attr_name FROM `iiimap_attraction`"

import time
Date = time.strftime('%Y%m%d',time.localtime(time.time()))
###　標籤結果
JFC='DB_OnlyConTags_'+Date+'.json'
JFW='DB_OnlyWordTags_'+Date+'.json'
FC='DB_OnlyConTags_'+Date+'.txt'
FW='DB_OnlyWordTags_'+Date+'.txt'

###　文字雲結果
JFCC='DB_FlatConceptCloud_'+Date+'.json'
JFWC='DB_FlatWordCloud_'+Date+'.json'
FCC='DB_FlatConceptCloud_'+Date+'.txt'
FWC='DB_FlatWordCloud_'+Date+'.txt'

###　所有JSON結果
JFid='DB_*'+Date+'.json'

###　收納檔案的資料夾名稱
Log1='Taggin_Log'+Date
Log2='Taggin_DBFiles'+Date
Log3='Taggin_DBTXT'+Date

###　其他參數
DBTXT='Taggin_DBTXT'+Date#　存放要丟進MySQL的TXT檔
LOGFid=Date+'_TagLog'#　存放全部的結果檔
DBfiles='DB_*'+Date+'.json'
DBTXTfiles='DB_*'+Date+'.txt'

###　DOS指令
CPCMD='cd Taggin_DBFiles'+Date+' & IF EXIST {} COPY {} ..'
MVCMD='IF EXIST {} MOVE {} '+DBTXT
DELCMD='cd '+DBTXT+' & IF EXIST {} DEL {}'
CollectCMD='IF EXIST {} MOVE {} '+LOGFid

In [2]:
import json
import MySQLdb
import os
import re
import subprocess

###　開新資料夾
os.popen('MKDIR '+DBTXT)

###　把檔案複製出來
subprocess.check_call(CPCMD.format(DBfiles,DBfiles),shell='TRUE')

###　連結資料庫
db = MySQLdb.connect(IP,User,PWd,DB,charset='utf8')#連結資料庫
cursor = db.cursor()

###　把JSON檔Load進來
Con = json.load(open(JFC,'r'))
Word= json.load(open(JFW,'r'))
ConC = json.load(open(JFCC,'r'))
WordC= json.load(open(JFWC,'r'))

F1= open(FC,'w')
F2= open(FW,'w')
F3= open(FCC,'w')
F4= open(FWC,'w')

###　把標籤.文字雲的結果轉成TXT檔
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        
        if row[1] in Con:#　如果景點名稱在Con裡面
            
            if len(Con[row[1]])>0:#　如果景點有標籤
                for i in Con[row[1]]:
                    ###　寫進檔案(每行)：景點編號,標籤
                    F1.write(str(row[0])+','+i.encode('utf-8')+'\n')
                
            if len(Word[row[1]])>0:#　如果景點有文字雲
                for i in Word[row[1]]:
                    ###　寫進檔案(每行)：景點編號,文字雲
                    F2.write(str(row[0])+','+i.encode('utf-8')+'\n')
except KeyError:
    print "Error: unable to fecth data"

###　把概念雲轉成TXT檔
for i in ConC:
    F3.write(i.encode('utf-8')+','+str(ConC[i])+'\n')
    
###　把文字雲轉成TXT檔
for i in WordC:
    F4.write(i.encode('utf-8')+','+str(WordC[i])+'\n')    

F1.close()
F2.close()
F3.close()
F4.close()

###　印出結果報告
print 'ConFile Len:',len([line for line in open(FC)])
print 'WordFile Len:',len([line for line in open(FW)])
print 'ConCloudFile Len:',len([line for line in open(FCC)])
print 'WordCloudFile Len:',len([line for line in open(FWC)])

###　把檔案移進Taggin_DBTXT的資料夾裡
subprocess.check_call(MVCMD.format(DBfiles,DBfiles),shell='TRUE')
subprocess.check_call(MVCMD.format(DBTXTfiles,DBTXTfiles),shell='TRUE')

###　把資料夾裡的JSON檔刪掉
subprocess.check_call(DELCMD.format(JFid,JFid),shell='TRUE')



###　把所有流程紀錄收起來　###

###　開資料夾
subprocess.check_call('MKDIR '+LOGFid,shell='TRUE')

###　依序收好
subprocess.check_call(CollectCMD.format(Log1,Log1),shell='TRUE')
subprocess.check_call(CollectCMD.format(Log2,Log2),shell='TRUE')
subprocess.check_call(CollectCMD.format(Log3,Log3),shell='TRUE')

ConFile Len: 1130
WordFile Len: 16967
ConCloudFile Len: 12
WordCloudFile Len: 9427


0